In [1]:
!pip install pytorch_lightning==0.7.6
!pip install comet-ml

     |████████████████████████████████| 256kB 2.7MB/s 
     |████████████████████████████████| 829kB 8.8MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=af9ed17c48b402aa745e3b0d9ab7a0d3908baeada386373187d65b175e7499e0
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
     |████████████████████████████████| 215kB 2.9MB/s 
     |████████████████████████████████| 204kB 8.7MB/s 
     |████████████████████████████████| 419kB 12.1MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=37b07f7415cccea7cfe191212660735bd00bba38f006ba8b3523bcb7c3a9ca8b
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jun 24 02:05:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Mount your personal google drive
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
primary_directory = '/content/drive/My Drive/KBP_Challenge'
%cd /content/drive/My\ Drive/KBP_Challenge
%ls

/content/drive/My Drive/KBP_Challenge
checkpoints/  data/  lightning_logs/  options/  provided_code/  src/


In [5]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.logging import CometLogger
from src.pix2pix_model import Pix2PixModel
from pytorch_lightning.callbacks import ModelCheckpoint
from options.train_options import TrainOptions

In [6]:
checkpoints_dir = '{}/checkpoints'.format(primary_directory)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoints_dir,
    verbose=True,
    monitor='loss',
    mode='min'
)
lr_logger = LearningRateLogger()

comet_logger = CometLogger(
    api_key="eyAsnp1KA7fXLxFMkEWKjhygS",
    project_name="kbp-challenge",
    workspace="mahruskazi"
)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Checkpoint directory /content/drive/My Drive/KBP_Challenge/checkpoints exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
CometLogger will be initialized in online mode


In [7]:
args = ['--batchSize', '8',
        '--which_model_netG', 'unet_128_3d',
        '--which_direction', 'AtoB',
        '--input_nc', '1',
        '--lambda_A', '100',
        '--lr_policy', 'plateau',
        '--epoch_count', '100',
        '--load_epoch', '-1',
        '--lr_decay_iters', '15000',
        '--lr', '0.01']

opt = TrainOptions().parse(args)

dataset_dir = '{}/data'.format(primary_directory)
model = Pix2PixModel(opt, dataset_dir, stage='training')

checkpoint = None
if opt.load_epoch != -1:
  checkpoint = '{}/epoch={}.ckpt'.format(checkpoints_dir, opt.load_epoch)

trainer = pl.Trainer(logger=comet_logger,
                     resume_from_checkpoint=checkpoint,
                     gpus=1,
                     checkpoint_callback=checkpoint_callback,
                     callbacks=[lr_logger],
                     max_epochs=opt.epoch_count,
                     check_val_every_n_epoch=3,
                     num_sanity_val_steps=5,
                     val_percent_check=0.5,
                     accumulate_grad_batches=1)
trainer.fit(model)

------------ Options -------------
batchSize: 8
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
dataroot: None
dataset_mode: unaligned
display_freq: 200
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 128
epoch_count: 100
fineSize: 128
gpu_ids: -1
init_type: normal
input_nc: 1
isTrain: True
lambda_A: 100.0
lambda_B: 10.0
lambda_identity: 0.5
lambda_vgg: 10.0
loadSize: 128
load_epoch: -1
lr: 0.01
lr_decay_iters: 15000
lr_policy: plateau
max_dataset_size: inf
model: pix2pix
nThreads: 2
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
niter: 100
niter_decay: 100
no_dropout: False
no_flip: False
no_html: False
no_img: False
no_lsgan: False
no_vgg_loss: False
norm: batch_3d
nwf: 64
output_nc: 1
phase: train
pool_size: 50
print_freq: 200
recon_loss: l1
resize_or_crop: resize_and_crop
save_epoch_freq: 5
save_latest_freq: 5000
serial_batches: False
training_size: 100
update_html_freq: 1000
wasserstein: False
which_direction: AtoB
which_epoch: l

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth


GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mahruskazi/kbp-challenge/d21457729f32424387cdc212e4b0798a


    | Name                                                                    | Type                    | Params
----------------------------------------------------------------------------------------------------------------
0   | generator                                                               | UnetGenerator           | 167 M 
1   | generator.model                                                         | UnetSkipConnectionBlock | 167 M 
2   | generator.model.model   

Number of validation patients: 99


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


100%|██████████| 5/5 [00:07<00:00,  1.55s/it]

{'dvh_score': 140.90068558760007, 'dose_score': 67.16820209305506}
Number of training patients: 100
Number of validation patients: 99



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


RuntimeError: ignored